In [3]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = "C:/Users/ANAND/Downloads/PENS_news_SEED.csv"

# Read the first 200 rows
data = pd.read_csv(file_path, nrows=200)

# Display the first few rows (optional)
print(data.head())


   Unnamed: 0 News ID Category         Topic  \
0           0  N10000   sports        soccer   
1           1  N10001     news  newspolitics   
2           2  N10002     news        newsus   
3           3  N10003     news  newspolitics   
4           4  N10004     news     newsworld   

                                            Headline  \
0  Predicting Atlanta United's lineup against Col...   
1  Mitch McConnell: DC statehood push is 'full bo...   
2            Home In North Highlands Damaged By Fire   
3  Meghan McCain blames 'liberal media' and 'thir...   
4                            Today in History: Aug 1   

                                           News body  \
0  Only FIVE internationals allowed, count em, FI...   
1  WASHINGTON -- Senate Majority Leader Mitch McC...   
2  NORTH HIGHLANDS (CBS13)   Fire damaged a home ...   
3  Meghan McCain is speaking out after a journali...   
4  1714: George I becomes King Georg Ludwig, Elec...   

                                     

In [4]:
# Save the data to 'train.csv'
data.to_csv('train.csv', index=False)

print("First 200 rows saved to 'train.csv'")

First 200 rows saved to 'train.csv'


In [6]:
train_data = pd.read_csv("train.csv")

In [7]:
train_data

,Unnamed: 0,News ID,Category,Topic,Headline,News body,T5_Seed
0,0,N10000,sports,soccer,Predicting Atlanta United's lineup against Col...,"Only FIVE internationals allowed, count em, FI...",[ 2.89325714e-01 5.46656772e-02 4.52104537e-...
1,1,N10001,news,newspolitics,Mitch McConnell: DC statehood push is 'full bo...,WASHINGTON -- Senate Majority Leader Mitch McC...,[ 5.33034615e-02 9.68789607e-02 5.80537580e-...
2,2,N10002,news,newsus,Home In North Highlands Damaged By Fire,NORTH HIGHLANDS (CBS13) Fire damaged a home ...,[-4.76725809e-02 3.65012020e-01 2.15179875e-...
3,3,N10003,news,newspolitics,Meghan McCain blames 'liberal media' and 'thir...,Meghan McCain is speaking out after a journali...,[-0.04048845 0.15008177 0.1026606 -0.131078...
4,4,N10004,news,newsworld,Today in History: Aug 1,"1714: George I becomes King Georg Ludwig, Elec...",[ 0.21329668 0.19950941 0.2605694 -0.206649...
...,...,...,...,...,...,...,...
195,195,N10195,travel,travelnews,Alaska Air criticized after video surfaces of ...,(Video by FOX News) Alaska Airlines is under f...,[ 8.17042887e-02 5.02950191e-01 6.90446049e-...
196,196,N10196,sports,basketball_ncaa,NBA Draft: Jaxson Hayes' mom was one of Iowa's...,Many NBA analysts suggest Texas power forward ...,[ 1.63968176e-01 6.36518747e-03 3.04574847e-...
197,197,N10197,sports,baseball_mlb,Alex Dickerson's historic debut leads Giants p...,PHOENIX The Giants called up outfielder Alex...,[-5.32138273e-02 2.16664985e-01 1.29206106e-...
198,198,N10198,news,newsus,3 Pounds Of Pot; New Playland Ride; Jersey Mik...,Share-worthy stories from the Hudson Valley Pa...,[ 4.40656900e-01 2.90401969e-02 -1.14445038e-...


In [ ]:
import torch
import pandas as pd
from r_tier import RTier  # Import RTier from the uploaded file
from r_cell import RCell  # Import RCell from the uploaded file

# Define dimensions (use appropriate values based on your model setup)
b_dim = 128  # Dimension of the behavior embeddings
h_dim = 64   # Dimension of the hidden state
r_dim = 32   # Dimension of the R cell output

# Load triplets and prepare input data
triplets_path = 'final_triplets_with_user.csv'  # Replace with the correct path if needed
triplets_df = pd.read_csv(triplets_path)

# Convert triplets into tensors
source_indices = torch.tensor(triplets_df['Source Index'].values, dtype=torch.long)
target_indices = torch.tensor(triplets_df['Target Index'].values, dtype=torch.long)
relation_indices = torch.tensor(triplets_df['Relation Index'].values, dtype=torch.long)

# Create embeddings for nodes and relations
num_nodes = max(source_indices.max().item(), target_indices.max().item()) + 1
num_relations = relation_indices.max().item() + 1

node_embeddings = torch.nn.Embedding(num_nodes, b_dim)
relation_embeddings = torch.nn.Embedding(num_relations, b_dim)

# Prepare input tensors
b_batch = node_embeddings(source_indices)  # Node embeddings for source nodes
r_batch = relation_embeddings(relation_indices)  # Relation embeddings

# Reshape tensors for the model
batch_size = 1  # Modify based on your batch setup
seq_len = len(source_indices)  # Total sequence length
b_batch = b_batch.view(batch_size, seq_len, -1)

# Add a placeholder for `is_summ` to match input signature (modify as needed)
is_summ = torch.zeros((batch_size, seq_len), dtype=torch.float32)  # Dummy values

# Initialize the RTier model
r_tier = RTier(b_dim, h_dim, r_dim)

# Run the model
outputs, hidden_state = r_tier((b_batch, is_summ, r_batch))

# Print outputs for verification
print("Outputs shape:", outputs.shape)
print("Hidden state shape:", hidden_state.shape)
